## Steps for generating object_detect.dlc

For this demo, a YoloNAS model is used. You can read more about this model in VisionSolution1-YoloNasSSD Readme.

**Installing Necessary Libraries**

In [ ]:
!pip3 install super-gradients==3.1.2
!pip3 install cython
!pip3 install yacs

### Getting the dataset

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017labels.zip -q --show-progress
!wget http://images.cocodataset.org/zips/val2017.zip -q --show-progress
!unzip val2017.zip
!unzip coco2017labels.zip


In [ ]:
import os
files = os.listdir('val2017')
for file in files[50:]:
    os.remove("val2017/"+file)

In [ ]:
%%bash
rm -rf coco
rm -rf coco2017labels.zip
rm -rf val2017.zip

#### Downloading the YOLO_Nas Model

In [ ]:
## Downloading Model from git repo
import torch
# Load model with pretrained weights
from super_gradients.training import models
from super_gradients.common.object_names import Models

model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")

# Prepare model for conversion
# Input size is in format of [Batch x Channels x Width x Height] where 640 is the standard COCO dataset dimensions
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, 320, 320])

# Create dummy_input
dummy_input = torch.randn([1, 3, 320, 320], device="cpu")

# Convert model to onnx
torch.onnx.export(model, dummy_input, "yolo_nas_s.onnx", opset_version=11)

#### Converting to DLC

In [ ]:
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/snpe/2.29.0.241129/"

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc -i yolo_nas_s.onnx -o ../app/src/main/assets/yolo_nas_s.dlc

## Quantizing Yolo_nas

In [ ]:
##STEPS to preprocess images

def preprocess(original_image):
    resized_image = cv2.resize(original_image, (320, 320))
    resized_image = resized_image/255
    return resized_image

import cv2
import numpy as np
import os


dataset_path = "val2017/"

os.makedirs('rawYoloNAS', exist_ok=True)

filenames=[]
for path in os.listdir(dataset_path)[:5]:
    # check if current path is a file
    if os.path.isfile(os.path.join(dataset_path, path)):
        filenames.append(os.path.join(dataset_path, path))

for filename in filenames:
    original_image = cv2.imread(filename)
    img = preprocess(original_image)
    img = img.astype(np.float32)
    img.tofile("rawYoloNAS/"+filename.split("/")[-1].split(".")[0]+".raw")

In [ ]:
%%bash
find rawYoloNAS -name *.raw > YoloInputlist.txt
cat YoloInputlist.txt

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-quantize --input_dlc ../app/src/main/assets/yolo_nas_s.dlc --input_list YoloInputlist.txt --output_dlc ../app/src/main/assets/Quant_yoloNas_s_320_online.dlc

snpe-dlc-graph-prepare requires device htp_soc info. 

So, depending on the device --htp_socs needs to be changed (sm8550 or sm8650 or sm8750)

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-graph-prepare --input_dlc ../app/src/main/assets/Quant_yoloNas_s_320_online.dlc --set_output_tensors 885,893 --output_dlc ../app/src/main/assets/Quant_yoloNas_s_320.dlc --htp_socs sm8750


## How to change the object-detect model ? 

Object detection models are highly dependant on model architecture, and the pre-processing requirements vary a lot from model to model. 
If user intends to use a different model e.g. YoloV5, following steps should be followed : 

- Ensure Qualcomm® Neural Processing SDK supports the operations in selected model
- Study the pre processing, and post processing requirements for the selected model
- Most object detection models operate in RGB space. Input camera YUV buffers need to be converted to RGB basd on model requirements 


# Info about HRNET

HRNET model is State-of-the-art model for human pose estimation. It has good accuracy for results with single person, but has lower accuracy for multiple persons. To enhance that, HRNET uses object-detect model to identify a single person in a frame and then give the data to HRNET to get pose of that person. In this solution, we use MobileNetSSD for detecting human and then give the preprocesssed data to HRNET to achieve better accuracy for pose estimation.

HRNET dlc takes 256x192x3 flattened array as input and returns output of dims 17x64x48. HRNET generates heatmap for 17 human joints and each heatmap is of size 64x48.

In [ ]:
%%bash
rm -rf HRNet-Human-Pose-Estimation/
git clone https://github.com/HRNet/HRNet-Human-Pose-Estimation.git
git checkout 00d7bf72f56382165e504b10ff0dddb82dca6fd2
cp hrnet.patch HRNet-Human-Pose-Estimation/
cd HRNet-Human-Pose-Estimation/
patch -p1 < ./hrnet.patch
cd lib
make

In [ ]:
%%bash
mkdir -p mode_binaries
cd mode_binaries
wget https://github.com/quic/aimet-model-zoo/releases/download/hrnet-posenet/hrnet_posenet_FP32.pth

In [ ]:
!pip install torch==1.11.0
!pip install torchvision==0.12.0

In [ ]:
#####################################################################
# Getting onnx from pth model for hrnet requires a different setup  #
# python 3.6                                                        #
# torch 1.10.1                                                      #
# torchvision 0.11.2                                                #
#####################################################################

import numpy as np
from matplotlib import pyplot as plt
import sys
import torch
import torch.utils.data
import torchvision.transforms as transforms
# from config import cfg
import os
import os.path as osp
import urllib.request

%matplotlib inline


lib_path = osp.join(os.getcwd(), 'HRNet-Human-Pose-Estimation/lib')
sys.path.insert(0, lib_path)
if not os.path.exists("model_binaries"):
    os.makedirs("model_binaries")
##Getting .pth file
OPTIMIZED_CHECKPOINT_URL = (
    # "https://github.com/quic/aimet-model-zoo/releases/download/hrnet-posenet/hrnet_posenet_FP32.pth"
    "https://github.com/quic/aimet-model-zoo/releases/download/hrnet-posenet/"
)

if not os.path.exists(f"./model_binaries/hrnet_posenet_FP32.pth"):
    urllib.request.urlretrieve(
        f"{OPTIMIZED_CHECKPOINT_URL}/hrnet_posenet_FP32.pth",
        f"model_binaries/hrnet_posenet_FP32.pth",
    )


input_shape = (1, 3, 256, 192)
dummy_input = torch.randn(input_shape)
model = torch.load("model_binaries/hrnet_posenet_FP32.pth")
model.to('cpu')

onnx_model_name = "model_binaries/AIMET_HRNET_posnet.onnx"

opset = 11

torch.onnx.export(
    model.cpu(),
    dummy_input,
    onnx_model_name,
    verbose=True,
    do_constant_folding=True,
    export_params=True,
    input_names=['input'],
    output_names=['output'],
    opset_version=opset
)


## Steps for generating HRNET dlc for int8

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc -i model_binaries/AIMET_HRNET_posnet.onnx -o ../app/src/main/assets/hrnet.dlc

## Steps for Quantization

In [ ]:
from PIL import Image
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)

preproc = transforms.Compose(
        [
            transforms.ToTensor(),
            normalize,
        ]
    )

In [ ]:

import cv2,os

dataset_path = "val2017/"

os.makedirs('rawHRNET', exist_ok=True)

filenames=[]
for path in os.listdir(dataset_path)[:5]:
    # check if current path is a file
    if os.path.isfile(os.path.join(dataset_path, path)):
        filenames.append(os.path.join(dataset_path, path))
print(filenames)

for filename in filenames:
    orig_img = cv2.imread(filename)
    img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(256,192),
                   interpolation = cv2.INTER_LINEAR)
    model_input = preproc(img).unsqueeze(0)

    model_input = model_input.cpu().detach().numpy()
    model_input = model_input.transpose(0,2,3,1)     
    fid = open("rawHRNET/"+filename.split("/")[-1].split(".")[0]+".raw", 'wb')
    model_input.tofile(fid)

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh

find rawHRNET -name *.raw > HRNET_input_list.txt
snpe-dlc-quantize --input_dlc ../app/src/main/assets/hrnet.dlc --input_list HRNET_input_list.txt --axis_quant --output_dlc ../app/src/main/assets/hrnet_axis_int8.dlc --enable_htp --htp_socs sm8750
snpe-dlc-info --input_dlc ../app/src/main/assets/hrnet_axis_int8.dlc > hrnet_axis_int8.txt